# Speed 5 (rev)

Challenge file is provided in [speed5](speed5).

Speed 1-4 were all incredibly speedy (solved in under half an hour), so there's no reason to expect this to be any different right? Right? (Hint: We were wrong.)

It took us about 2.5 hours to solve this challenge, albeit with a breakfast break in between. Anyway, the first thing to note is that you don't really get that much "actual logic" from IDA Pro (or any other disassembler/decompiler really). This is because it is quite a barebones VM, and really the only thing it does it to invoke a function in the VM called VERIFY. So we have to look at the code that gets parsed by the VM instead.

Well, we can take a shortcut and just look at the ASCII dump of the data section, as it has function names littered all around. We also place red lines to distinguish what we believe are different segments (mainly because it comes right before a LAMBDA):

![speed_dump2](speed_dump2.png)

The last section shows the eight functions that are defined natively: MAX, DIV, CONS, CDR, CAR, NAND, MUL, SUB. Most of these are easily inferable, and we note that [CAR and CDR](https://en.wikipedia.org/wiki/CAR_and_CDR) refers to the head and tail of a Lisp linked list. I have zero experience with Lisp, but have seen similar formulations in F# and Haskell so it wasn't completely unfamiliar territory.

The rest are functions defined within the VM, some of these you can reasonably infer what they might do from the name:
* OR(x, y) = NAND(NOT(x), NOT(y))
* NOT(x) = NAND(x, x)
* ADD(x, y) = SUB(x, SUB(0, y))
* MOD(x, y) = SUB(x, MUL(DIV(x,y), y))
* FOLD(...) is a bit harder to read due to the IF, but it very likely does a [fold](https://en.wikipedia.org/wiki/Fold_(higher-order_function))

Which leaves VERIFY, HASH, HELPER, MASSAGE, and LOLWTF. We can make some reasonable guesses from these:
* VERIFY(x,y) = if( OR(SUB(HASH(x), ???), SUB(HASH(y), ???) )) then ???
* HASH(x) = fold the HELPER function over LOLWTF(x)
* MASSAGE(x) = MOD(MUL(SUB(MAX(x, ???), ???), ???), ???)
* HELPER(x) = ADD(MASSAGE(MUL(x, ???), ???), ???)
* LOLWTF = ???????

These are very vague guesses, but at this stage it would greatly help us to hook into the SUB function and record all subtractions. Assuming we've inferred it correctly, then we are simply comparing a function of our string against a target hash. We use Qiling to do this, by hooking at 0x2264:

![speed_dump3](speed_dump3.png)

In [1]:
from qiling import Qiling
from qiling.const import QL_VERBOSE

# run with our input string, and record the two arguments everytime we hit 0x2264
def get_subtract_pairs(s):
    def check(ql):
        arg1 = ql.arch.regs.rax
        arg2 = ql.unpack(ql.mem.read(ql.arch.regs.rdx, 8))
        foo.append((arg1, arg2))
    foo = []
    ql = Qiling(['speed5', s], 'x8664_linux', verbose=QL_VERBOSE.OFF)
    ql.hook_address(check, 0x2264 + ql.loader.load_address)
    ql.run()
    return foo

[(hex(a),hex(b)) for a,b in get_subtract_pairs('test')]

[('0x4000000000000074', '0x400000000000005f'),
 ('0x4000000000000111', '0x40000000000000f8'),
 ('0x4000000000000000', '0x4000000000000000'),
 ('0x4000000000000019', '0x4000000000000000'),
 ('0x4000000000000065', '0x400000000000005f'),
 ('0x400000000000004e', '0x400000000000003e'),
 ('0x4000000000000000', '0x4000000000000307'),
 ('0x4000000000000010', '0x7ffffffffffffcf9'),
 ('0x4000000000000317', '0x468a49b35ef1a677'),
 ('0x4000000000000073', '0x400000000000005f'),
 ('0x4000000000000104', '0x40000000000000f8'),
 ('0x4000000000000000', '0x4000000000000000'),
 ('0x400000000000000c', '0x4000000000000000'),
 ('0x4000000000000074', '0x400000000000005f'),
 ('0x4000000000000111', '0x40000000000000f8'),
 ('0x4000000000000000', '0x4000000000000174'),
 ('0x4000000000000019', '0x7ffffffffffffe8c'),
 ('0x400000000000018d', '0x4a9ee44a73efff1e')]

In [2]:
# Let's try a shorter string next?
[(hex(a),hex(b)) for a,b in get_subtract_pairs('a')]

[('0x4000000000000000', '0x468a49b35ef1a677'),
 ('0x4000000000000061', '0x400000000000005f'),
 ('0x400000000000001a', '0x4000000000000000'),
 ('0x4000000000000000', '0x4000000000000000'),
 ('0x400000000000001a', '0x4000000000000000'),
 ('0x400000000000001a', '0x4a9ee44a73efff1e')]

A number of things strike out at us almost immediately:
* The second highest bit is always set (which is why you have to AND with 0x3FFFFFFFFFFFFFFF)
* The highest bit is sometimes set? This is presumably part of an ADD
* There are two random-looking values that are constant, these are presumably the target hashes

In any case, let's see if we can produce the hashes for a number of inputs.

In [3]:
def test(x):
    arr = get_subtract_pairs(x)
    h1 = next(a&0x3FFFFFFFFFFFFFFF for a,b in arr if b==0x468a49b35ef1a677)
    h2 = next(a&0x3FFFFFFFFFFFFFFF for a,b in arr if b==0x4a9ee44a73efff1e)
    print(f'{x}: {h1}, {h2}')
    
test('a')
test('aa')
test('aaa')
test('aaaa')
test('aaaaa')
test('b')
test('bb')
test('bbb')
test('bbbb')
test('bbbbb')
test('ab')
test('ba')
test('c')
test('d')
test('e')
test('A')
test('B')
test('midnight')

a: 0, 26
aa: 26, 26
aaa: 26, 832
aaaa: 832, 832
aaaaa: 832, 25818
b: 0, 8
bb: 8, 8
bbb: 8, 256
bbbb: 256, 256
bbbbb: 256, 7944
ab: 8, 26
ba: 26, 8
c: 0, 21
d: 0, 3
e: 0, 16
A: 0, 0
B: 0, 0
midnight: 755631, 720870


The natural thing to infer from this is that longer strings have larger hashes, and it looks like it splits the strings into the two halves, with the first half being shorter if necessary.

We notice that uppercase letters don't do anything, and the lowercase ones go 26 -> 8 -> 21 -> 3 -> 16, which is an addition of 13 (mod 31). Likewise, we notice that:
* 832 = 26 \* (31 + 1)
* 25818 = 26 \* (31^2 + 31 + 1)
* 256 = 8 \* (31^2 + 31 + 1)
* 7944 = 8 \* (31^2 + 31 + 1)

So that it is just a base-31 string, which each letter multiplied by 13 (then adding some offset).

In [4]:
def split_base31(n): 
    return [n%31] + split_base31(n//31) if n else []

def decode_base31(n):
    return bytes((i*pow(13,-1,31)-2)%31+97 for i in split_base31(n))

print(decode_base31(755631)) # expect 'midn'
print(decode_base31(720870)) # expect 'ight'

b'ingt'
b'mdih'


Well, we messed up the theory slightly. It's not exactly halves, but interleaved letters.

In [5]:
s1 = decode_base31(0x468a49b35ef1a677&0x3FFFFFFFFFFFFFFF)
s2 = decode_base31(0x4a9ee44a73efff1e&0x3FFFFFFFFFFFFFFF)
print(s1)
print(s2)
print(b''.join(bytes([a,b]) for a,b in zip(s2,s1)))

b'ingtseddsre}'
b'mdih{pe~iodr'
b'midnight{speed~disorder}'


That looks like a valid flag! Except that `~` is ASCII 126 and `_` is ASCII 95 (so congruent mod 31). And we had arbitrarily assumed that it begins with ASCII 97 (`a`), when it really probably begins at 95 (cannot be 94 since we need `}`).

The flag is thus `midnight{speed_disorder}`.